In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('train.csv')
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [0]:
train = df[['effectiveness_rating', 'number_of_times_prescribed',	'base_score']]
train.head()

,effectiveness_rating,number_of_times_prescribed,base_score
0,9,27,8.022969
1,8,192,7.858458
2,5,17,6.341969
3,9,37,6.590176
4,2,43,6.144782


In [0]:
#sample
train = train.sample(frac=1).reset_index(drop=True)

In [0]:
train.corr()

,effectiveness_rating,number_of_times_prescribed,base_score
effectiveness_rating,1.000000,0.242224,0.412301
number_of_times_prescribed,0.242224,1.000000,0.134295
base_score,0.412301,0.134295,1.000000


In [0]:
x = train[['effectiveness_rating',	'number_of_times_prescribed']]
y = train[['base_score']]

In [0]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(x)  # fit does nothing.
transformer

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
x_norm = transformer.transform(x)

In [0]:
from sklearn import tree
clf = tree.DecisionTreeRegressor(criterion='friedman_mse')
clf.fit(x, y)

DecisionTreeRegressor(ccp_alpha=0.5, criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
clf

DecisionTreeRegressor(ccp_alpha=0.5, criterion='friedman_mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x)

In [0]:
y_pred

array([7.50975766, 5.85995262, 5.85995262, ..., 7.50975766, 5.85995262,
       5.85995262])

In [0]:
df_test = pd.read_csv('test.csv')
df_test.head()

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2


In [0]:
predict_on = df_test[['effectiveness_rating', 'number_of_times_prescribed']]

In [0]:
z = clf.predict(predict_on)

In [0]:
submit = pd.DataFrame(columns=['patient_id', 'base_score'])

In [0]:
submit['patient_id'] = df_test['patient_id']
submit.head()

,patient_id,base_score
0,163740,NaN
1,39293,NaN
2,208087,NaN
3,23295,NaN
4,97013,NaN


In [0]:
submit['base_score'] = z
submit.head()

,patient_id,base_score
0,163740,7.509758
1,39293,7.509758
2,208087,5.859953
3,23295,5.859953
4,97013,5.859953


In [0]:
submit.to_csv('dt_fmse_ccp.csv', index=False)

-------------------------------------------------------------------

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.22, random_state=42)

In [0]:
pd.set_option("display.precision", 8)

In [0]:
import lightgbm as lgb 

In [0]:
predict_on.to_numpy()

array([[10, 22],
       [ 9, 35],
       [ 4, 13],
       ...,
       [10, 43],
       [ 8,  7],
       [ 9, 46]])

In [0]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor

rfr = AdaBoostRegressor()#xgb.XGBRegressor( max_depth=55, n_estimators=340, tree_method='exact',reg_lambda=50.2999, learning_rate=0.308999, base_score=0.3999, booster='dart') 
#(max_depth=len(y), random_state=0, criterion='friedman_mse', oob_score=True)
#rfr = lgb.LGBMRegressor()#(max_depth=100, n_estimators=320, tree_method='exact',reg_lambda=50.2999, learning_rate=0.308999, base_score=0.3999, grow_policy="lossguide")
rfr.fit(x_norm, y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

In [0]:
#train score     320
rfr.score(x_norm, y)
#0.9999973513904648
#0.99997/8xxx

#0.9999817462011953 gamma almost zero
#0.9999820730474576 gamma zero <- 2nd best till now
#0.999982123378497 on using dart new best

0.4954054159911493

In [0]:
#rfr.get_params()

In [0]:
z = rfr.predict(transformer.transform(predict_on))

In [0]:
submit = pd.DataFrame(columns=['patient_id', 'base_score'])
submit.head()

,patient_id,base_score


In [0]:
submit['patient_id'] = df_test['patient_id']
submit.head()

,patient_id,base_score
0,163740,NaN
1,39293,NaN
2,208087,NaN
3,23295,NaN
4,97013,NaN


In [0]:
submit['base_score'] = z
submit.head()

,patient_id,base_score
0,163740,7.49603464
1,39293,7.50126496
2,208087,6.85047422
3,23295,4.48332986
4,97013,6.85047422


In [0]:
submit['base_score'] = submit['base_score'].round(5)
#submit.base_score = submit.base_score * 100000
#submit.base_score = submit.base_score.apply(np.floor)
#submit.base_score = submit.base_score / 100000
submit.head()

,patient_id,base_score
0,163740,8.75726986
1,39293,8.74501991
2,208087,5.95239019
3,23295,5.99464989
4,97013,5.35319996


In [0]:
submit.to_csv('ada.csv', index=False)
#rl2 s> rl1

again ne 340>320

md60 --> 94.59951<br>
md50 --> 94.4<br>
md55 --> 94.60658

4-4-20 08:02 -->
94.51306  ab se sab round karo

In [0]:
#randomness
for i, data in submit.iterrows():
  r1 = np.random.randint(1,9) / 100000
  r2 = np.random.randint(1,9) / 10000
  r3 = np.random.randint(1,9) / 1000000
  r = r1 + r2 +r3
  new = data.base_score + r
  submit.at[i, 'base_score'] = new


ab toh saturate ho chuka hai dedo 1st prize yaar

mds2 0 was better

spw no effect as of now

* bs 0.3999 ++9 --> 94.52141
* bs 0.38 --> 94.34
* bs 0.4 --> 94.51306
* bs 0.44 --> 94.43
* bs 0.41 --> 94.42
* bs 0.3 --> 94.44

lr 0.308999 --> 94.48306<br>
lr 0.301 --> 94.461<br>
lr 0.3 --> 94.4<br>
lr 0.4 --> 94.37<br>

round 4 --> 94.28203
<br>rl 50.2999 ==> 94.282<br>
rl 50.29 ==> 94.237<br>
rl 50.3 ==> 94.264<br>
rl 50.333 ==> 94.20<br>

rl2 slight improvement<br/>
rl3 2% more(91) <br>
rl30 93<br>
rl50 94.25<br>
rl50.9 94.22<br>
--55 94.12<br>
--53 94.16<br>
rl70 reduced to 93

---
adding year too

In [0]:
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [0]:
year = df['drug_approved_by_UIC'].apply(lambda x: str(x)[-2:])
year.unique()

array(['12', '10', '09', '16', '15', '17', '14', '11', '08', '13'],
      dtype=object)

In [0]:
x1 = train[['effectiveness_rating',	'number_of_times_prescribed']]
y1 = train[['base_score']]
x1['year'] = year.astype(int)
x1.head()

,effectiveness_rating,number_of_times_prescribed,year
0,10,65,12
1,1,18,10
2,6,7,9
3,3,5,16
4,2,5,15


In [0]:
t1 = StandardScaler().fit(x1)
x1_norm = t1.transform(x1)

In [0]:
rfr1 = xgb.XGBRegressor(max_depth=55, n_estimators=320, tree_method='exact',reg_lambda=50.2999, learning_rate=0.308999, base_score=0.3999, grow_policy="lossguide")
#(max_depth=len(y), random_state=0, criterion='friedman_mse', oob_score=True)
rfr1.fit(x1_norm, y1)
rfr1.score(x1_norm, y1)

[01:29:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9999844038644705

In [0]:
predict_on1 = df_test[['effectiveness_rating', 'number_of_times_prescribed']]
pyear = df_test['drug_approved_by_UIC'].apply(lambda x: str(x)[-2:])
predict_on1['year'] = pyear.astype(int)
predict_on1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,effectiveness_rating,number_of_times_prescribed,year
0,10,22,12
1,9,35,17
2,4,13,14
3,7,21,16
4,2,44,15


In [0]:
z1 = rfr1.predict(t1.transform(predict_on1))
submit1 = pd.DataFrame(columns=['patient_id', 'base_score'])
submit1['patient_id'] = df_test['patient_id']
submit1['base_score'] = z
submit1.head()

,patient_id,base_score
0,163740,8.75723648
1,39293,8.74540901
2,208087,5.95182228
3,23295,5.99488306
4,97013,5.35705662


In [0]:
submit1.to_csv('00d-best.csv', index=False)

---

In [0]:
#############################################################################

In [0]:
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [0]:
x_new = pd.concat([x, predict_on])
x_new.shape

(42925, 2)

In [0]:
y_new = y.to_numpy()
y_new = np.append(y_new, submit.base_score.to_numpy())
y_new.shape

(42925,)

In [0]:
new_df = x_new
new_df['base_score'] = y_new
new_df.head()

,effectiveness_rating,number_of_times_prescribed,base_score
0,10,65,8.25088977
1,1,18,4.02755433
2,6,7,2.39755614
3,3,5,7.40841870
4,2,5,6.57728564


In [0]:
from sklearn.utils import shuffle
seed = 99
snew_df = new_df.sample(frac=1).reset_index(drop=True)
snew_df.head()

,effectiveness_rating,number_of_times_prescribed,base_score
0,10,12,6.35586554
1,7,17,5.52171704
2,9,28,8.81147275
3,10,12,6.35586554
4,10,27,7.92190695


In [0]:
rftar = xgb.XGBRegressor(max_depth=55, n_estimators=1500, tree_method='exact',reg_lambda=50.2999, learning_rate=0.308999, base_score=0.3999, ) 
xx = snew_df[['effectiveness_rating', 'number_of_times_prescribed']]
yy = snew_df[['base_score']]
t1 = StandardScaler().fit(xx)
nx_new = t1.transform(sx_new)


rftar.fit(nx_new, yy)
rftar.score(nx_new, yy)

[01:47:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.035694852357916584

In [0]:
nx_new

array([[-0.30232447, -0.78620912],
       [-1.82906302, -0.72968535],
       [-0.30232447, -0.61663781],
       ...,
       [ 0.00302324,  0.51383762],
       [ 0.00302324,  0.79645648],
       [ 0.61371866,  0.88124214]])

In [0]:
y_new.head()

,base_score
6466,0.899076
62,6.169374
18449,1.268007
7029,NaN
2563,7.351884


In [0]:
z_new = rftar.predict(t1.transform(predict_on))

In [0]:
z_new

array([6.5051913, 6.8430877, 6.6427903, ..., 6.793213 , 6.9593496,
       6.778131 ], dtype=float32)

In [0]:
submit_ch = pd.DataFrame(columns=['patient_id', 'base_score'])
submit_ch['patient_id'] = submit['patient_id']
submit_ch['base_score'] = z_new
submit_ch.to_csv('patanahi.csv', index=False)

Ek Alag hi concept pata nhi kaam karega ya nhi
--
par dekhte hain

In [0]:
ex = 